In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import random
import re
import pandas as pd 

In [2]:
# WebDriver 설정
options = webdriver.ChromeOptions()
options.add_argument("window-size=1920x1080")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument(
    "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
    "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# 스크롤 로직
def scroll_to_bottom(scrollable_element):
    last_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_element)
    while True:
        driver.execute_script("arguments[0].scrollTop += 600;", scrollable_element)
        time.sleep(1)
        new_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_element)
        if new_height == last_height:
            break
        last_height = new_height

# 네이버 지도 접속
driver.get("https://map.naver.com/")
time.sleep(3)

# 검색어 입력 및 검색
search_box = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CLASS_NAME, "input_search"))
)
search_box.send_keys("강서구 맛집")
search_box.send_keys(Keys.ENTER)
time.sleep(5)

# iframe 전환
try:
    iframe = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "iframe#searchIframe"))
    )
    driver.switch_to.frame(iframe)
except Exception as e:
    print(f"iframe 전환 오류: {e}")
    driver.quit()
    exit()

# 결과 리스트 초기화
results = []
visited_urls = set()
page_number = 1
MAX_PAGES = 5
MAX_RESULTS = 150  # 최대 결과 수 제한

while page_number <= MAX_PAGES and len(results) < MAX_RESULTS:
    try:
        # 스크롤을 맨 아래까지 내리기
        scrollable_element = driver.find_element(By.CLASS_NAME, "Ryr1F")
        scroll_to_bottom(scrollable_element)

        # 현재 페이지의 가게 리스트 가져오기
        store_elements = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, "place_bluelink"))
        )
        print(f"현재 {page_number}페이지) 총 {len(store_elements)}개의 가게를 찾았습니다.")

        for store_index in range(len(store_elements)):
            if len(results) >= MAX_RESULTS:
                print("최대 결과 수에 도달하여 크롤링을 종료합니다.")
                break

            try:
                # 가게 리스트를 다시 가져와 클릭
                store_elements = driver.find_elements(By.CLASS_NAME, "place_bluelink")
                store_elements[store_index].click()
                time.sleep(random.uniform(3, 5))

                # 상세 iframe 전환
                driver.switch_to.default_content()
                detail_iframe = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "iframe#entryIframe"))
                )
                driver.switch_to.frame(detail_iframe)

                # 페이지 소스 파싱
                html = driver.page_source
                soup = BeautifulSoup(html, "lxml")

                # 가게 정보 추출
                place_name = soup.select_one("span.GHAhO").text.strip() if soup.select_one("span.GHAhO") else "N/A"
                category = soup.select_one("span.lnJFt").text.strip() if soup.select_one("span.lnJFt") else "N/A"
                address = soup.select_one("span.LDgIH").text.strip() if soup.select_one("span.LDgIH") else "N/A"
                current_url = driver.current_url
                res_code = re.findall(r"place/(\d+)", current_url)
                store_url = f'https://pcmap.place.naver.com/restaurant/{res_code[0]}/review/visitor#' if res_code else "N/A"

                # 중복 여부 확인
                if store_url not in visited_urls:
                    visited_urls.add(store_url)
                    results.append({
                        "가게 이름": place_name,
                        "업종구분": category,
                        "주소": address,
                        "URL": store_url
                    })
                    print(f"{len(results)}. {place_name}")

                # iframe 복귀
                driver.switch_to.default_content()
                iframe = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "iframe#searchIframe"))
                )
                driver.switch_to.frame(iframe)

            except Exception as e:
                print(f"가게 크롤링 중 오류 발생: {e}")
                continue

        if len(results) >= MAX_RESULTS:
            print("최대 결과 수에 도달하여 크롤링을 종료합니다.")
            break

        # '다음' 버튼 클릭
        if page_number < MAX_PAGES:
            try:
                next_button = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, '//a[@class="eUTV2" and @aria-disabled="false"]'))
                )
                driver.execute_script("arguments[0].click();", next_button)
                print(f"'다음' 버튼 클릭: 페이지 {page_number + 1}로 이동 중")
                time.sleep(random.uniform(5, 10))

                # iframe 전환
                driver.switch_to.default_content()
                iframe = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "iframe#searchIframe"))
                )
                driver.switch_to.frame(iframe)

            except Exception as e:
                print(f"'다음' 버튼 클릭 중 오류 발생: {e}")
                break

        # 페이지 번호 증가
        page_number += 1

    except Exception as e:
        print(f"페이지 크롤링 중 오류 발생: {e}")
        break

# CSV 파일 저장
file_name = "Gangseo-gu_restaurants.csv"
df = pd.DataFrame(results)
df.to_csv(file_name, index=False, encoding="utf-8-sig")

print(f"크롤링 완료. 결과는 {file_name}에 저장되었습니다.")
driver.quit()

현재 1페이지) 총 76개의 가게를 찾았습니다.
1. 돈구멍 마곡본점
2. 모루야
3. 면옥담 마곡본점
4. 송쭈집 마곡
5. 원조장족발 등촌점
6. 봄담아 마곡점
7. 우니크
8. 봉이밥
9. 어나더사이드
10. 만배아리랑보쌈 본점
11. 마곡관
12. 탐라팩토리 마곡발산점
13. 광주똑순이아구찜
14. 만배아리랑보쌈 등촌점
15. 일편등심 마곡발산점
16. 봄담아 등촌점[본점]
17. 모미지 마곡발산점
18. 윤가네 역전오리 마곡발산점
19. 호호방
20. 추돈가 발산본점
21. 해탄 발산마곡 본점
22. 석암생소금구이 마곡본점
23. 오마카세 오사이초밥 발산역점
24. 여수갓육고 발산본점
25. 일품한우 마곡본점
26. 화창 마곡본점
27. 굴사냥화곡본동점
28. 일번지호프
29. 옥소반 마곡본점
30. 담솥 마곡점
31. 가능식당 마곡나루역점
32. 한가네숯불닭갈비 강서구청점
33. 범식당 까치산본점
34. 피그백
35. 탄천갈매기 마곡점
36. 깊고푸른참치
37. 온샐러드앤포케 마곡역점
38. 취링취링 화곡역점
39. 유림 닭도리탕
40. 고릴라쉐프 마곡 본점
41. 모로미 강서구청점
42. 카루
43. 규카츠정 마곡점
44. 더메이드뷔페
45. 양마니곱창
46. 산청숯불가든 마곡
47. 을왕리꾸덕집 마곡직영점
48. 초이다이닝 마곡점
49. 소앤돼지 마곡나루점
50. 대소상회 까치산점
51. 장지녕간장게장 김포공항점
52. 두꺼비숙성횟집 마곡직영점
53. 등촌 최월선칼국수
54. 금고깃집 마곡본점
55. 오유미당 마곡점
56. 마장동김씨 우장산역점
57. 요쇼쿠 마곡점
58. 직화장인 마곡점
59. 생굴사랑
60. 베이커리 룬
61. 마곡그랑한우
62. 담산 마곡점
63. 별곱창 까치산본점
64. 페페네로
65. 타이반쩜 마곡점
66. 1982논현황소곱창 본점
67. 도니랑김치생삼겹 마곡나루역점
68. 샤브20 발산마곡점
69. 도일처
70. 우대포 마곡점
71. 진부령황태촌
72. 프레베리 마곡케이크
73. 철길부산집 발산점
74. 화곡영양족발
7